## Домашнее задание №2 (курс "Практикум по программированию на языке Python")

### Тема: Объектно-ориентированное программирование на языке Python

#### Преподаватель: Мурат Апишев (mel-lain@yandex.ru)

**Выдана**:   11 марта 2023

**Дедлайн**:   21:00 26 марта 2023

**Среда выполнения**: Jupyter Notebook (Python 3.7)

#### Правила:

Результат выполнения задания - Jupyter Notebook с кодом и подробными ответами в случае теоретических вопросов. __Максимальное число баллов за задание - 20__.

Все ячейки должны быть "выполненными", при этом результат должен воспроизводиться при проверке (на Python 3.7). Если какой-то код не был запущен или отрабатывает с ошибками, то пункт не засчитывается. Задание, сданное после дедлайна, _не принимается_. Можно отправить недоделанное задание, выполненные пункты будут оценены.

Готовое задание отправляется на почту преподавателя.

Задание выполняется самостоятельно. Если какие-то студенты будут уличены в списывании, все они автоматически получат за эту работу 0 баллов. Если вы нашли в Интернете какой-то специфичный код, который собираетесь заимствовать, обязательно укажите это в задании - наверняка вы не единственный, кто найдёт и использует эту информацию.

Удалять фрагменты формулировок заданий запрещается.

In [1]:
!python --version

Python 3.7.5


#### Постановка задачи:

- В данной работе нужно
    - ответить на ряд теоретических вопросов;
    - решить набор задач, проверяющих владение ООП-инструментами языка;
    - решить задачу на проектирование кода.
- Ответы на теоретические вопросы должны быть полными и обоснованными.
- Каждая задача представляет собой написание функции или класса, а также набора тестов, проверяющих работу решения в общих и крайних случаях.
- Отсутствие тестов автоматически уменьшает количество баллов за задание как минимум в два раза, некачественные тесты также будут штрафоваться.
- Даже если это не указано явно в требованиях, код должен быть по возможности неизбыточным, работать с разумной сложностью и объёмом потребялемой памяти, проверяющие могут снизить балл за задание, выполненное без учёта этого требования.
- Результирующий код должен быть читаемым, с единой системой отступов и адеквантными названиями переменных, проверяющие могут снизить балл за задание, выполненное без учёта этого требования.

__Задание 1 (2 балла):__ Дайте подробные ответы на следующие вопросы:

1. В чём смысл инкапсуляции? Приведите пример конкретной ситуации в коде, в которой нарушение инкапсуляции приводит к проблемам.
2. Какой метод называется статическим? Что такое параметр `self`?
3. В чём отличия методов `__new__` и `__init__`?
4. Какие виды отношений классов вы знаете? Для каждого приведите примеры. Укажите взаимные различия.
5. Зачем нужны фабрики? Опишите смысл использования фабричного метода, фабрики и абстрактной фабрики, а также их взаимные отличия.

1. Смысл инкапсуляции заключается в том, чтобы запретить изменять состояние объектов класса извне.  

In [138]:
# Пример нарушения инкапсуляции

class Client:
    def __init__(self, name, balance):
        self.name = name
        self.balance = balance  # баланс клиента
    
    def cache_out(self, value):
        """ Снятие наличных """
        if value <= self.balance:
            self.balance -= value

# В данном примере приведен класс, который хранит персональные данные клиента банка.
# При такой реализации можно изменить баланс клиента извне:

client = Client('Vova', 100) 

client.balance *= 100
print(client.balance)

# Также можно погасить задолженность, хотя функция предназначена для 
# использования внутри других методов

client.cache_out(5000)
print(client.balance)

10000
5000


In [139]:
# Решением проблемы является инкапсулирование при помощи двух нижних подчеркиваний. 
# Одинарная черточка является скорее сигналом для других разработчиков. Возможность изменять состояние 
# класса остается.

class Client:
    def __init__(self, name, balance):
        self._name = name
        self.__balance = balance
        
    
    def __cache_out(self, value):
        if value <= self.__balance:
            self.__balance -= value
    
    def try_to_cache_out(self, value, pin):
        if pin == '0000':
            self.__cache_out(value)
    
    @property
    def get_balance(self):
        return self.__balance
            
client = Client('Vova', 100)

client._name = 'Not Vova'
print(client._name)

try:
    client.__balance *=10000
except AttributeError:
    print('Error')
    
try:
    client.__cache_out(5000)
except AttributeError:
    print('Error')
    
client.try_to_cache_out(50, '0000')
print(client.get_balance)

Not Vova
Error
Error
50


Стоит отметить, что инкапсуляция используется скорее для защиты от случайного использования "внутренних методов" в коде, чем от реальной защиты "секретных данных".

2. Статический метод - метод, который определен внутри класса, но не имеет доступа к его экземплярам. Он не может обратиться к атрибутам экземпляра или изменить их состояние. self - ссылка на конкретный экземпляр класса.

3. __new__ - статический метод класса, он создает и возвращает новый экземпляр класса. __new__ вызывается до метода __init__. __init__ используется для инициализации созданного объекта. Метод __new__ является первым методом, который вызывается при создании нового экземпляра класса. Резюмируя: __new__ используется для создания экземпляра класса, а __init__ используется для инициализации созданного экземпляра.

4. Существует три типа отношений между классами: наследование, композиция и агрегация.

- Наследование - отношение между двумя классами, где один класс является наследником другого(базового). Наследование позволяет создавать новый класс, который наследует функциональность и может расширять ее или изменять поведение.

- Композиция - отношение, где объект одного класса является частью объекта другого класса. Композиция позволяет создавать сложные объекты, объединяя их из более простых объектов.

- Агрегация - отношение, где один класс имеет ссылку на другой класс, но не является его частью. В отличие от композиции, объекты классов могут существовать независимо друг от друга.

5. Фабрики используются для создания объектов без необходимости напрямую вызывать конструкторы. Это позволяет упростить код и сделать его более гибким. Фабричный метод позволяет подклассам решать, какие классы создавать, фабрика создает объекты разных типов в зависимости от переданных параметров, а абстрактная фабрика предоставляет интерфейс для создания связанных объектов и предоставляет одну или несколько конкретных реализаций для создания объектов.

In [328]:
# Наследование
class Animal:
    def __init__(self, name):
        self.name = name

    def make_sound(self):
        pass

class Dog(Animal):
    def make_sound(self):
        return 'Woof!'

In [329]:
# Композиция
class Engine:
    def __init__(self, power):
        self.power = power

class Car:
    def __init__(self, model, engine: Engine):
        self.model = model
        self.engine = engine

In [330]:
# Агрегация

from typing import List

class Student:
    def __init__(self, name):
        self.name = name

class Classroom:
    def __init__(self, name, students: List[Student]):
        self.name = name
        self.students = students

__Задание 2 (1 балл):__ Опишите класс комплексных чисел. У пользователя должна быть возможность создать его объект на основе числа и в алгебраической форме, и в полярной. Класс должен поддерживать основные математические операции (+, -, \*, /) за счет перегрузки соответствующих магических методов. Также он должен поддерживать возможность получить число в алгебраической и полярной форме. Допускается использование модуля `math`.

In [149]:
import math


class ComplexNumberException(Exception):
    def __init__(self, msg):
        self.message = msg

    
class ComplexNumber:
    def __init__(self, re=None, im=None, r=None, theta=None):
        if r is not None and theta is not None:
            self.re = r * math.cos(theta)/1.0
            self.im = r * math.sin(theta)/1.0
        elif re is not None and im is not None:
            self.re = re/1.0
            self.im = im/1.0
        else:
            raise ComplexNumberException('Неправильная форма')
    
    @property
    def get_algebraic_form(self):
        sym = '+' if self.im >= 0 else ''
        return f'{self.re}{sym}{self.im}i'

    @property
    def get_polar_form(self):
        r = math.sqrt(self.re**2 + self.im**2)
        theta = math.atan2(self.im, self.re)
        
        return (r, theta)
    
    def __add__(self, other):
        return ComplexNumber(self.re + other.re, self.im + other.im)
    
    def __sub__(self, other):
        return ComplexNumber(self.re - other.re, self.im - other.im)
    
    def __mul__(self, other):
        return ComplexNumber(self.re**2 - self.im**2, self.im*other.re + self.re*other.im)
    
    def __truediv__(self, other):
        x_1, y_1 = self.re, self.im
        x_2, y_2 = other.re, other.im
        
        try:
            _re = (x_1*x_2 + y_1*y_2) / (x_2**2 + y_2**2)
            _im = (x_2*y_1 - x_1*y_2) / (x_2**2 + y_2**2)
        except ZeroDivisionError:
            raise ComplexNumberException('На 0 делить опасно!')
            
        return ComplexNumber(_re, _im)
    
    def __eq__(self, other):
        return self.re == other.re and self.im == self.im
    
    def __round__(self):
        return ComplexNumber(round(self.re, 2), round(self.im, 2))

In [165]:
def test_complex_number():
    zero_number = ComplexNumber(re=0, im=0)

    not_valid_numbers = [
        {'r': 1, 'im': 2},
        {'re': 0, 'theta': 30},
        {'re': 1},
        {'im': 0},
        {'r': 0},
        {'theta': 0},
    ]
    
    # Проверка инициализации числа
    for num in not_valid_numbers:
        try:
            cn = ComplexNumber(**num)
        except ComplexNumberException:
            print(f'OK for validation invalid number {num}')

    # Проверка деления на 0
    try:
        res = zero_number / zero_number
    except ComplexNumberException:
        print(f'OK for zero division')
        
    num_1 = ComplexNumber(re=1, im=0)
    num_2 = ComplexNumber(re=2, im=-3)
    num_3 = ComplexNumber(r=3, theta=30)
    num_4 = ComplexNumber(r=3, theta=0)
    
    # Проверка сложения
    assert num_1 + num_2 == ComplexNumber(re=3, im=3)
    assert round(num_2 + num_3) == ComplexNumber(re=2.46, im=-5.96)
    assert round(num_3 + num_4) == ComplexNumber(re=3.46, im=-2.96)
    print('OK for __add__')
    
    # Проверка вычитания
    assert round(num_1 - num_2) == ComplexNumber(re=-1, im=3)
    assert round(num_2 - num_3) == ComplexNumber(re=1.54, im=-0.04)
    assert round(num_3 - num_4) == ComplexNumber(re=-2.54, im=-2.96)
    print('OK for __sub__')
    
    # Проверка умножения
    assert round(num_1 * num_2) == ComplexNumber(re=1, im=-3)
    assert round(num_2 * num_3) == ComplexNumber(re=-5, im=-7.3)
    assert round(num_3 * num_4) == ComplexNumber(re=-8.57, im=-8.89)
    print('OK for __sub__')
    
    # Проверка деления
    assert round(num_1 / num_2) == ComplexNumber(re=0.15, im=0.23)
    assert round(num_2 / num_3) == ComplexNumber(re=1.09, im=0.51)
    assert round(num_3 / num_4) == ComplexNumber(re=0.15, im=-0.99)
    print('OK for __truediv__')
        
test_complex_number()

OK for validation invalid number {'r': 1, 'im': 2}
OK for validation invalid number {'re': 0, 'theta': 30}
OK for validation invalid number {'re': 1}
OK for validation invalid number {'im': 0}
OK for validation invalid number {'r': 0}
OK for validation invalid number {'theta': 0}
OK for zero division
OK for __add__
OK for __sub__
OK for __sub__
OK for __truediv__


__Задание 3 (2 балла):__ Опишите класс для векторов в N-мерном пространстве. В качестве основы  используйте список значений координат вектора, задаваемый `list`. Обеспечьте поддержку следующих операций: сложение, вычитание (с созданием нового вектора-результата), скалярное произведение, косинус угла, евклидова норма. Все операции, которые можно перегрузить с помощью магических методов, должны быть реализованы именно через них. Класс должен производить проверку консистентности аргументов для каждой операции и в случаях ошибок выбрасывать исключение `ValueError` с исчерпывающим объяснением ошибки.

In [171]:
import math


class VectorException(Exception):
    def __init__(self, msg):
        self.message = msg

        
class Vector:
    def __init__(self, vector_values_list):
        self.vector_values_list = vector_values_list

    def __len__(self):
        return len(self.vector_values_list)

    def __getitem__(self, i):
        return self.vector_values_list[i]

    def __setitem__(self, i, value):
        self.vector_values_list[i] = value

    def __add__(self, other):
        if len(self) != len(other):
            raise VectorException('Нельзя складывать векторы разной длины')
        return Vector([self[i] + other[i] for i in range(len(self))])

    def __sub__(self, other):
        if len(self) != len(other):
            raise VectorException('Нельзя вычитать векторы разной длины')
        return Vector([self[i] - other[i] for i in range(len(self))])

    def dot(self, other):
        if len(self) != len(other):
            raise VectorException('Нельзя посчитать скалярное произведение для векторов разной длины')
        return sum([self[i] * other[i] for i in range(len(self))])

    def cos(self, other):
        if len(self) != len(other):
            raise VectorException('Невозможно вычислить косинус угла между векторами разной длины')
        dot_product = self.dot(other)
        magnitude_product = math.sqrt(self.dot(self) * other.dot(other))
        return dot_product / magnitude_product

    def norm(self):
        return math.sqrt(sum([x ** 2 for x in self.vector_values_list]))

    def __str__(self):
        return str(self.vector_values_list)
    
    def __eq__(self, other):
        for i in range(len(self.vector_values_list)):
            if self.vector_values_list != other.vector_values_list:
                return False
        return True

In [178]:
def test_vector():
    v1 = Vector([1, 2, 3])
    v2 = Vector([4, 5, 6])

    assert v1 + v2 == Vector([5, 7, 9])
    assert v2 + v1 == Vector([5, 7, 9])

    assert v1 - v2 == Vector([-3, -3, -3])
    assert v2 - v1 == Vector([3, 3, 3])

    assert v1.dot(v2) == 32
    assert v2.dot(v1) == 32

    assert math.isclose(v1.cos(v2), 0.9746318461970762)
    assert math.isclose(v2.cos(v1), 0.9746318461970762)

    assert math.isclose(v1.norm(), 3.7416573867739413)
    assert math.isclose(v2.norm(), 8.774964387392123)

    assert v1[0] == 1
    assert v2[2] == 6

    v1[0] = 4
    assert v1 == Vector([4, 2, 3])

    # проверяем длину вектора
    assert len(v1) == 3
    assert len(v2) == 3
    
    print('OK')

test_vector()

OK


__Задание 4 (2 балл):__ Опишите декоратор, который принимает на вход функцию и при каждом её вызове печатает строку "This function was called N times", где N - число раз, которое это функция была вызвана на текущий момент (пока функция существует как объект, это число, очевидно, может только неубывать).

In [180]:
import functools

def calls_counter(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        wrapper.num_calls += 1
        print(f'This function was called {wrapper.num_calls} times')
        return func(*args, **kwargs)
    wrapper.num_calls = 0
    return wrapper

In [194]:
def test_calls_counter():
    @calls_counter
    def test_func(a, b):
        pass
    
    @calls_counter
    def test_func_1():
        pass
    
    test_func(1, 2)
    test_func_1()
    test_func(1, b=2)
    test_func_1()
    test_func(a=1, b=2)
    test_func_1()
    
    assert test_func.num_calls == 3
    assert test_func_1.num_calls == 3
    
    print('OK')
    
test_calls_counter()

This function was called 1 times
This function was called 1 times
This function was called 2 times
This function was called 2 times
This function was called 3 times
This function was called 3 times
OK


__Задание 5 (3 балла):__ Опишите декоратор класса, который принимает на вход другой класс и снабжает декорируемый класс всеми атрибутами входного класса, названия которых НЕ начинаются с "\_". В случае конфликтов имён импортируемый атрибут должен получить имя с суффиксом "\_new".

In [320]:
def copy_class_attrs(cls):
    def decorator(decorated_class):
        # Примечание: фактически, методы в Python - это атрибуты класса, которые являются функциями,
        # поэтому имена методов тоже учитываем.
        for name, value in cls.__dict__.items(): 
            if not name.startswith('_'):
                if not hasattr(decorated_class, name):
                    setattr(decorated_class, name, value)
                else:
                    setattr(decorated_class, f'{name}_new', value)
        return decorated_class
    return decorator

In [321]:
def test_copy_class_attrs():
    class Foo:
        a = 1
        _b = 2

        def c(self):
            pass
        
        def _d(self):
            pass
        
    @copy_class_attrs(cls=Foo)
    class Boo:
        a = 3

    boo = Boo()
    assert boo.a == 3
    assert boo.a_new == 1
    assert hasattr(boo, 'c')
    
    print('OK')
    
test_copy_class_attrs()

OK


__Задание 6 (7 баллов):__ Опишите класс для хранения двумерных числовых матриц на основе списков. Реализуйте поддержку индексирования, итерирования по столбцам и строкам, по-элементные математические операции (с помощью магических методов), операцию умножения матрицы (как метод `dot` класса), транспонирование, поиска следа матрицы, а также поиск значения её определителя, если он существует, в противном случае соответствующий метод должен выводить сообщение об ошибке и возвращать `None`.

Матрицу должно быть возможным создать из списка (в этом случае у неё будет одна строка), списка списков, или же передав явно три числа: число строк, число столбцов и значение по-умолчанию (которое можно не задавать, в этом случае оно принимается равным нулю). Все операции должны проверять корректность входных данных и выбрасывать исключение с информативным сообщением в случае ошибки.

Матрица должна поддерживать методы сохранения на диск в текстовом и бинарном файле и методы обратной загрузки с диска для обоих вариантов. Также она должна поддерживать метод полного копирования. Обе процедуры должны быть реализованы с помощью шаблона "примесь" (Mixin), т.е. указанные функциональности должны быть описаны в специализированных классах.

В реализации математических операций запрещается пользоваться любыми функциями, требующими использования оператора `import`.

In [211]:
class MatrixException(Exception):
    def __init__(self, msg):
        self.message = msg

In [212]:
import pickle

class MatrixMixin:
    def save_text(self, filename):
        with open(filename, 'w') as f:
            for row in self.matrix:
                for element in row:
                    f.write(str(element) + ' ')
                f.write('\n')

    def save_binary(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump(self, f)

    @staticmethod
    def load_text(filename):
        with open(filename, 'r') as f:
            lines = f.readlines()
            data = [[int(x) for x in line.split()] for line in lines]
            return Matrix(data)

    @staticmethod
    def load_binary(filename):
        with open(filename, 'rb') as f:
            return pickle.load(f)

In [354]:
import copy

class Matrix(MatrixMixin):
    def __init__(self, *args):
        if len(args) == 1:
            if isinstance(args[0], list):
                if args[0] == []:
                    self.rows = 0
                    self.cols = 0
                    self.matrix = args[0]
                elif isinstance(args[0][0], list):   
                    self.rows = len(args[0])
                    self.cols = len(args[0][0])
                    self.matrix = args[0]
                elif isinstance(args[0][0], (int, float)):   
                    self.rows = 1
                    self.cols = len(args[0])
                    self.matrix = [args[0]]
            else:
                raise MatrixException('Неправильно задана матрица')
        elif len(args) == 2:
            if isinstance(args[0], int) and isinstance(args[1], int):
                self.rows = args[0]
                self.cols = args[1]
                self.matrix = [[0 for _ in range(self.cols)] for _ in range(self.rows)]
            else:
                raise MatrixException('Неправильно задана матрица')
        elif len(args) == 3:
            self.rows = args[0]
            self.cols = args[1]
            self.matrix = [[args[2] for _ in range(self.cols)] for _ in range(self.rows)]
        else:
            raise MatrixException('Слишком много параметров')
            
    def __getitem__(self, index):
        row, col = index
        if row < 0 or row >= self.rows or col < 0 or col >= self.cols:
            raise IndexError('Index out of range')
        return self.matrix[row][col]
    
    def __setitem__(self, index, value):
        row, col = index
        self.matrix[row][col] = value
    
    def __iter__(self):
        self.row = 0
        return self
    
    def __next__(self):
        if self.row == self.rows:
            raise StopIteration
        row = self.matrix[self.row]
        self.row += 1
        return row
    
    def columns(self):
        return zip(*self.matrix)
            
    def transpose(self):
        """ Транспонирование матрицы """
        self.matrix = [list(i) for i in zip(*self.matrix)]
        
    def trace(self):
        """ Поиск следа матрицы """
        if self.rows != self.cols:
            raise MatrixException('След можно вычислить только для квадратной матрицы')
        return sum(self.matrix[i][i] for i in range(len(self.matrix)))
    
    def determinant(self):
        """ Поиск определителя """
        
        def _minor(matr, i, j):
            matr_copy = copy.deepcopy(matr)
            del matr_copy[i]
            for i in range(len(matr[0]) - 1):
                del matr_copy[i][j]
            return matr_copy   

        def _det(matr):
            rows_num = len(matr)
            cols_num = len(matr[0])
            if rows_num != cols_num:
                return None
            if cols_num == 1:
                return matr[0][0]
            
            signum = 1
            determinant = 0

            for j in range(cols_num):
                determinant += matr[0][j] * signum * _det(_minor(matr, 0, j)) 
                signum *= -1
                
            return determinant  
        
        return _det(self.matrix)
    
    
    def dot(self, other):
        """ Умножение матриц """
        
        if isinstance(other, int) or isinstance(other, float):
            result = Matrix(self.rows, self.cols)
            for i in range(self.rows):
                for j in range(self.cols):
                    result.matrix[i][j] = self.matrix[i][j] * other
            return result
        elif isinstance(other, Matrix) and self.cols == other.rows:
            result = Matrix(self.rows, other.cols)
            for i in range(self.rows):
                for j in range(other.cols):
                    result.matrix[i][j] = sum([self.matrix[i][k] * other.matrix[k][j] for k in range(self.cols)])
            return result
        else:
            raise MatrixException('Невозможно перемножить матрицы с такими размерностями')

    def __add__(self, other):
        """ Сложение матриц """
        if isinstance(other, Matrix) and self.rows == other.rows and self.cols == other.cols:
            result = [[0 for i in range(self.cols)] for j in range(self.rows)]
            for i in range(self.rows):
                for j in range(self.cols):
                    result[i][j] = self.matrix[i][j] + other.matrix[i][j]
            return Matrix(result)
        else:
            raise MatrixException('Нельзя складывать матрицы разной размерности')

    def __sub__(self, other):
        """ Вычитание матриц """
        if isinstance(other, Matrix) and self.rows == other.rows and self.cols == other.cols:
            result = Matrix(self.rows, self.cols, 0)
            for i in range(self.rows):
                for j in range(self.cols):
                    result.matrix[i][j] = self.matrix[i][j] - other.matrix[i][j]
            return result
        else:
            raise MatrixException('Нельзя вычитать матрицы разной размерности')
            
    def __eq__(self, other):
        if isinstance(other, Matrix) and self.rows == other.rows and self.cols == other.cols:
            for i in range(self.rows):
                for j in range(self.cols):
                    if self.matrix[i][j] != other.matrix[i][j]:
                        return False
            return True
        else:
            return False
        
    
    def __str__(self):
        return '\n'.join(['\t'.join([str(item) for item in row]) for row in self.matrix])
    
    def __repr__(self):
        return '\n'.join(['\t'.join([str(item) for item in row]) for row in self.matrix])

In [358]:
def test_matrix():
    
    m0 = Matrix([[5, 6], [0, 8]])
    
    # Создание матрицы
    m1 = Matrix([])
    m2 = Matrix([[1, 2], [3, 4]])
    m3 = Matrix([1, 2, 3, 4])
    m4 = Matrix(2, 3, 1)
    
    # Индексирование
    assert m0[1, 0] == 0
    try:
        m0[2, 2]
    except IndexError:
        pass
    
    m0[1, 0] = 7
    assert m0[1, 0] == 7
    
    # Итерирование по строкам и столбцам
    r = [r_i for r_i in m0]
    assert r == [[5, 6], [7, 8]]
    c = [c_i for c_i in m0.columns()]
    assert c == [(5, 7), (6, 8)]
    
    # Поэлементные операции
    assert m0+m2 == Matrix([[6, 8], [10, 12]])
    assert m0-m2 == Matrix([[4, 4], [4, 4]])
    
    # След матрицы
    m7 = Matrix([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
    assert m7.trace() == 15
    
    # Определитель 
    assert m7.determinant() == 0
    assert m2.determinant() == -2
    assert not m3.determinant()
    
    print('OK')
    
test_matrix()

OK


__Задание 7 (3 балла):__ Ставится задача расчета стоимости чашки кофе. Опишите классы нескольких типов кофе (латте, капучино, американо), а также классы добавок к кофе (сахар, сливки, кардамон, пенка, сироп). Используйте шаблон "декоратор". Каждый класс должен характеризоваться методом вычисления стоимости чашки `calculate_cost`. Пользователь должен иметь возможность комбинировать любое число добавок с выбранным кофе и получить на выходе общую стоимость:

```
Cream(Sugar(Latte())).calculate_cost()
```

Первым элементом чашки всегда должен быть сам кофе, а не добавка, в противном случае при попытке создания чашки должно выбрасываться исключение:

```
Cream(Latte(Sugar())).calculate_cost() -> exception
```

Кофе может встречаться в чашке только один раз, в противном случае при попытке создания чашки должно выбрасываться исключение:

```
Cappuccino(Sugar(Latte())).calculate_cost() -> exception
```

Добавки могут включаться в чашку в любом количестве и порядке.
Добавление новых типов кофе и добавок не должно требовать изменения существующего кода.

In [171]:
from abc import ABC, abstractmethod

class Coffee(ABC):
    """ Базовый класс """
    
    @abstractmethod
    def calculate_cost(self):
        pass

In [364]:
# Классы для каждого типа коффе

class Latte(Coffee):
    def __init__(self, coffee=None):
        if coffee:
            raise TypeError('Кофе может встречаться в чашке только один раз')
    
    def calculate_cost(self):
        return 150

class Cappuccino(Coffee):
    def __init__(self, coffee=None):
        if coffee:
            raise TypeError('Кофе может встречаться в чашке только один раз')
    
    def calculate_cost(self):
        return 100

class Americano(Coffee):
    def __init__(self, coffee=None):
        if coffee:
            raise TypeError('Кофе может встречаться в чашке только один раз')
    
    def calculate_cost(self):
        return 50

In [365]:
# Классы для каждого типа добавки

class Cream(Coffee):
    
    def __init__(self, coffee):
        if not isinstance(coffee, Coffee):
            raise TypeError('Первым элементом в чашке должен быть коффе')
        self.coffee = coffee
    
    def calculate_cost(self):
        return self.coffee.calculate_cost() + 25

class Sugar(Coffee):
    
    def __init__(self, coffee):
        if not isinstance(coffee, Coffee):
            raise TypeError('Первым элементом в чашке должен быть коффе')
        self.coffee = coffee
    
    def calculate_cost(self):
        return self.coffee.calculate_cost() + 5

class MilkFoam(Coffee):
    
    def __init__(self, coffee):
        if not isinstance(coffee, Coffee):
            raise TypeError('Первым элементом в чашке должен быть коффе')
        self.coffee = coffee
    
    def calculate_cost(self):
        return self.coffee.calculate_cost() + 10

class Cardamom(Coffee):
    
    def __init__(self, coffee):
        if not isinstance(coffee, Coffee):
            raise TypeError('Первым элементом в чашке должен быть коффе')
        self.coffee = coffee
    
    def calculate_cost(self):
        return self.coffee.calculate_cost() + 12

class Syrup(Coffee):
    
    def __init__(self, coffee):
        if not isinstance(coffee, Coffee):
            raise TypeError('Первым элементом в чашке должен быть коффе')
        self.coffee = coffee
    
    def calculate_cost(self):
        return self.coffee.calculate_cost() + 20

In [372]:
def test_cup_maker():
    
    cup1 = Cream(Sugar(Latte()))
    assert cup1.calculate_cost() == 180

    cup2 = Cardamom(MilkFoam(Americano()))
    assert cup2.calculate_cost() == 72

    cup3 = Syrup(MilkFoam(Sugar(Cappuccino())))
    assert cup3.calculate_cost() == 135
    
    try:
        Cream(Sugar())
    except TypeError:
        pass
    else:
        raise AssertionError
    
    try:
        Latte(Cream(Sugar()))
    except TypeError:
        pass
    else:
        raise AssertionError
        
    try:
        Latte(Cream(Sugar(Latte())))
    except TypeError:
        pass
    else:
        raise AssertionError
    
    print('OK')
    
test_cup_maker()

OK
